In [15]:
import os

from DeepPurpose.dataset import *
import DTI_inspire as models
import matplotlib.pyplot as plt
import pandas as pd
from utils_inspire import *
from torch.utils.data import SequentialSampler
from prettytable import PrettyTable

train = pd.read_pickle('data/G35_data_train.pkl')
val = pd.read_pickle('data/G35_data_vali.pkl')
test = pd.read_pickle('data/G35_data_test.pkl')


model = models.model_pretrained(path_dir="models/second_trial_run")

In [16]:
model

In [17]:
model.config

{'input_dim_drug': 2586,
 'input_dim_protein': 8420,
 'hidden_dim_drug': 128,
 'hidden_dim_protein': 128,
 'cls_hidden_dims': [1024, 1024, 512],
 'batch_size': 64,
 'train_epoch': 100,
 'test_every_X_epoch': 20,
 'LR': 0.0001,
 'drug_encoding': 'Transformer',
 'target_encoding': 'CNN_inspire',
 'result_folder': './result/',
 'binary': False,
 'num_workers': 8,
 'transformer_emb_size_drug': 128,
 'transformer_num_attention_heads_drug': 8,
 'transformer_intermediate_size_drug': 512,
 'transformer_n_layer_drug': 8,
 'transformer_dropout_rate': 0.1,
 'transformer_attention_probs_dropout': 0.1,
 'transformer_hidden_dropout_rate': 0.1,
 'inspire_activation': 'elu',
 'CNN_inspire_filters': 128,
 'protein_strides': [10, 15, 20, 25, 30],
 'inspire_dropout': 0,
 'protein_layers': [128],
 'decay': 0}

In [20]:
model.model.cuda()
model.model.eval()

info = data_process_loader(test.index.values, test.Label.values, test, **model.config)
params_test = {'batch_size': model.config['batch_size'],
        'shuffle': False,
        'num_workers': model.config['num_workers'],
        'drop_last': False,
        'sampler':SequentialSampler(info)}

if (model.drug_encoding == "MPNN"):
    params_test['collate_fn'] = mpnn_collate_func
testing_generator = data.DataLoader(data_process_loader(test.index.values, test.Label.values, test, **model.config),         **params_test)

float2str = lambda x:'%0.4f'%x

mse, r2, p_val, CI, logits = model.test_(testing_generator, model.model)
test_table = PrettyTable(["MSE", "Pearson Correlation", "with p-value", "Concordance Index"])
test_table.add_row(list(map(float2str, [mse, r2, p_val, CI])))
print('Testing MSE: ' + str(mse) + ' , Pearson Correlation: ' + str(r2) 
    + ' with p-value: ' + str(p_val) +' , Concordance Index: '+str(CI))
np.save(os.path.join(model.result_folder, str(model.drug_encoding) + '_' + str(model.target_encoding) 
    + '_logits.npy'), np.array(logits))    

Testing MSE: 1.3332898783180298 , Pearson Correlation: 0.3028579974570897 with p-value: 9.310653684039609e-164 , Concordance Index: 0.6342781730294978
